# Codes for living park dataset prepproc
1. Running this code to test whether new T1 SERIES description appears and also validate the subject session labels for additional livingPark subjects from metadata accompanied the images (.xml file for each image).


In [14]:
# from pathlib import Path
import sys
from pathlib import Path
import glob
import os
import shutil
import numpy as np
import pandas as pd

# system PATH
code_path_str='/scratch/mr_proc/'
sys.path.append(code_path_str)

# study PATH
code_dir = Path(code_path_str)
xml_path = Path('/scratch/0tmp/meta_data')
xml_out_path = Path(code_dir / 'metadata'  / 'PPMI_livingpark2_metadata.csv')

In [15]:
# read meta data
import xml.etree.ElementTree as ET

files = [f for f in os.listdir(xml_path) if f[-3:]=='xml']
tree = ET.parse(str(xml_path/files[0]))
root = tree.getroot()
subject_info_list= [];
for file in files:
    tree = ET.parse(str(xml_path/file))
    root = tree.getroot()
    data_dict={};
    data_dict['Image Data ID']=root[0][3][4][6][0].text
    data_dict['Subject']=root[0][3][0].text
    data_dict['Group']=root[0][3][1].text
    data_dict['Sex']=root[0][3][2].text
    data_dict['Visit']=root[0][3][3][0].text
    data_dict['Age']=root[0][3][4][1].text
    data_dict['Modality']=root[0][3][4][5][1].text
    data_dict['Description']=root[0][3][4][6][1].text
    data_dict['Acq Date']=root[0][3][4][5][2].text
    subject_info_list.append(data_dict)
df=pd.DataFrame(subject_info_list)
df

,Image Data ID,Subject,Group,Sex,Visit,Age,Modality,Description,Acq Date
0,1537582,110955,Prodromal,M,Baseline,68.56,MRI,3D T2 SAG FLAIR_ND,2022-01-18
1,1537581,110955,Prodromal,M,Baseline,68.56,MRI,3D T2 SAG FLAIR,2022-01-18
2,1537571,110955,Prodromal,M,Baseline,68.56,MRI,AX T2 GRE MT,2022-01-18
3,1537572,110955,Prodromal,M,Baseline,68.56,MRI,AX T2 GRE MT,2022-01-18
4,1537573,110955,Prodromal,M,Baseline,68.56,MRI,AX T2 GRE MT,2022-01-18
...,...,...,...,...,...,...,...,...,...
591,1348316,75423,Prodromal,M,Baseline,72.08,MRI,Cal Head 24,2018-10-25
592,1199314,75447,Prodromal,M,Baseline,63.59,MRI,AXIAL PD-T2 TSE FS,2018-12-10
593,1199315,75447,Prodromal,M,Baseline,63.59,MRI,AXIAL PD-T2 TSE FS,2018-12-10
594,1199316,75447,Prodromal,M,Baseline,63.59,MRI,MPRAGE_GRAPPA_ADNI,2018-12-10


In [16]:
# T1 sequence description,copied from heuristics.
T1W_SERIES = [
    'MPRAGE 2 ADNI',
    'MPRAGE ADNI',
    'MPRAGE GRAPPA 2',
    'MPRAGE GRAPPA2',
    'MPRAGE GRAPPA2(adni)',
    'MPRAGE w/ GRAPPA',
    'MPRAGE_GRAPPA',
    'MPRAGE_GRAPPA_ADNI',
    'MPRAGE GRAPPA',
    'SAG T1 3D MPRAGE',
    'sag mprage',
    'MPRAGEadni',
    'MPRAGE GRAPPA_ND',
    '3D SAG',
    'MPRAGE T1 SAG',
    'MPRAGE SAG',
    'SAG T1 3DMPRAGE',
    'SAG T1 MPRAGE',
    'SAG 3D T1',
    'SAG MPRAGE GRAPPA2-NEW2016',
    'SAG MPRAGE GRAPPA_ND',
    'Sag MPRAGE GRAPPA',
    'AXIAL T1 3D MPRAGE',
    'SAG MPRAGE GRAPPA',
    'sT1W_3D_FFE',
    'sT1W_3D_ISO',
    'sT1W_3D_TFE',
    'sag 3D FSPGR BRAVO straight',
    'SAG T1 3D FSPGR',
    'SAG FSPGR 3D '
    'SAG 3D FSPGR BRAVO STRAIGHT',
    'SAG T1 3D FSPGR 3RD REPEAT',
    'SAG FSPGR BRAVO',
    'SAG SPGR 3D',
    'SAG 3D SPGR',
    'FSPGR 3D SAG',
    'SAG FSPGR 3D',
    'SAG 3D FSPGR BRAVO STRAIGHT',
    'SAG FSPGR 3D ',
    't1_mpr_ns_sag_p2_iso',
    'T1',
    'T1 Repeat',
    'AX T1',
    'axial spgr',
    'T1W_3D_FFE AX',
    # added by Vincent
    'AX T1 SE C+',
    '3D SAG T1 MPRAGE',
    '3D SAG T1 MPRAGE_ND',
    '3D T1',
    '3D T1 MPRAGE',
    '3D T1-weighted',
    'Accelerated Sag IR-FSPGR',
    'MPRAGE',
    'MPRAGE - Sag',
    'MPRAGE Phantom GRAPPA2',
    'MPRAGE w/ GRAPPA 2',
    'PPMI_MPRAGE_GRAPPA2',
    'SAG 3D T1 FSPGR',
    'SAG FSPGR 3D VOLUMETRIC T1',
    'Sag MPRAGE GRAPPA_ND',
    'T1-weighted, 3D VOLUMETRIC',
    'tra_T1_MPRAGE',
    '3D T1-weighted_ND', ## added from livingpark
    '3D T1 _weighted',
    'Sagittal 3D Accelerated MPRAGE',
    'T1 REPEAT',
    'MPRAGE Repeat',
    'SAG_3D_MPRAGE'
]

In [17]:
# test wheter there are new T1 SERIES in coming data
print("Mannully check whether there is new T1 sequences:")
display(set(df.Description.unique())-set(T1W_SERIES))
# Logs: 
# for livingpark1, new serise has added;
# for livingpark2, no new series added;

Mannully check whether there is new T1 sequences:


{'*AX FSE T2',
 '2D GRE MT MTC-NO',
 '2D GRE-MT',
 '3D T2 FLAIR',
 '3D T2 SAG FLAIR',
 '3D T2 SAG FLAIR_ND',
 'AX DE TSE',
 'AX DUAL_TSE',
 'AX FLAIR',
 'AX FLAIR T2',
 'AX GRE -MT',
 'AX PD + T2',
 'AX T2',
 'AX T2 DE',
 'AX T2 FLAIR',
 'AX T2 GRE MT',
 'AXIAL  T2  FSE',
 'AXIAL FLAIR',
 'AXIAL PD-T2 TSE FS',
 'Ax PD /T2',
 'Ax T2 FLAIR',
 'Ax T2 FSE with Fat Sat',
 'Axial FLAIR',
 'Axial PD-T2 TSE',
 'Axial PD-T2 TSE FS',
 'Axial PD-T2-FS TSE',
 'Axial T2 TSE with Fat Sat',
 'Axial T2-FLAIR',
 'BOLD_RS ACPC_LINE',
 'Cal Head 24',
 'Cor FLAIR TI_2800ms',
 'DTI_LR',
 'DTI_RL',
 'FLAIR',
 'FLAIR AXIAL',
 'FLAIR_LongTR AX',
 'L>R RESTING STATE FMRI ep2d_fid_basic_bold',
 'PD-T2 DUAL AXIAL TSE',
 'PPMI_AX_T2_FLAIR',
 'PPMI_Axial T2 TSE with Fat Sat',
 'R>L RESTING STATE FMRI ep2d_fid_basic_bold',
 'SAG 3D FLAIR',
 'SAG 3D MPRAGE',
 'SAG 3D T2 FLAIR_',
 'T2',
 'repeat_BOLD_RS ACPC_LINE',
 'rsfMRI_LR',
 'rsfMRI_RL'}

In [ ]:
df=pd.DataFrame(subject_info_list)
def encode_visit(x_):
    if x_=='Baseline':
        return 1 # 
    elif x_=='Screening':
        return 0
    else:
        return -1
df.loc[:,'Visit'] = [ encode_visit(x) for x in df['Visit'] ]
df_baseline=df[df['Visit']>=0].copy()
#df["T1 include"] = [1 if x in T1W_SERIES  else 0 for x in df['Description']]
#t1_df=df[df["T1 include"]==1].copy()
#t1_df=t1_df.drop(columns=['T1 include'])
df_baseline.loc[:,"Image Data ID"] = ["I"+str(x) for x in df_baseline['Image Data ID']]
#df=df.apply(lambda x: "\""+str(x)+"\"")
print('Final size: ', df.shape)
# save file
#xml_out_path = Path(code_dir / 'metadata'  / 'PPMI_livingpark2_metadata.csv')
df_baseline.to_csv(xml_out_path, index=False)

Final size:  (596, 9)
